# Final Project for COGS118A, this is meant to find and identify russian 'troll' tweets.

Group members :
- Gael Van der Lee
- Alex Labranche

In [1]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import random
import numpy as np
from urllib.request import urlopen
from datetime import timezone, datetime

# Data gathering and cleaning

### Loading and cleaning

In [3]:
# Load the 3 datasets
troll_users = pd.read_csv('Tweets/Trolls/users.csv')
troll_tweets = pd.read_csv('Tweets/Trolls/tweets.csv')
normal_tweets = pd.read_csv('Tweets/Normal/dashboard_x_usa_x_filter_nativeretweets.csv')

In [4]:
print('Columns of each dataframe at import :\n\nTroll users : {}\n\nTroll tweets : {}\n\nNormal tweets : {}'
      .format(troll_users.columns.values, troll_tweets.columns.values, normal_tweets.columns.values))

Columns of each dataframe at import :

Troll users : ['id' 'location' 'name' 'followers_count' 'statuses_count' 'time_zone'
 'verified' 'lang' 'screen_name' 'description' 'created_at'
 'favourites_count' 'friends_count' 'listed_count']

Troll tweets : ['user_id' 'user_key' 'created_at' 'created_str' 'retweet_count'
 'retweeted' 'favorite_count' 'text' 'tweet_id' 'source' 'hashtags'
 'expanded_urls' 'posted' 'mentions' 'retweeted_status_id'
 'in_reply_to_status_id']

Normal tweets : ['Tweet Id' 'Date' 'Hour' 'User Name' 'Nickname' 'Bio' 'Tweet content'
 'Favs' 'RTs' 'Latitude' 'Longitude' 'Country' 'Place (as appears on Bio)'
 'Profile picture' 'Followers' 'Following' 'Listed'
 'Tweet language (ISO 639-1)' 'Tweet Url']


In [5]:
# Make Nicknames consistent
troll_users['screen_name'] = troll_users['screen_name'].str.lower()

# Drop the irrelevant data
users_to_drop = ['statuses_count', 'time_zone', 'verified', 'favourites_count']
tweets_to_drop = ['created_at', 'tweet_id', 'source', 'posted', 'retweeted_status_id', 'retweeted', 'in_reply_to_status_id']
normal_to_drop = ['Tweet Id', 'Latitude', 'Longitude', 'Country', 'Profile picture', 'Tweet Url']
troll_users = troll_users.drop(users_to_drop, axis=1)
troll_tweets = troll_tweets.drop(tweets_to_drop, axis=1)
normal_tweets = normal_tweets.drop(normal_to_drop, axis=1)

In [6]:
users_new_cols = {'id': 'User Id', 'location': 'Location', 'name': 'User Name', 
                  'followers_count': 'Followers', 'lang': 'Language', 'screen_name': 'Nickname',
                  'description': 'Bio', 'created_at': 'Account creation date', 
                  'friends_count': 'Following', 'listed_count': 'Listed'}
tweets_new_cols = {'user_id': 'User Id', 'user_key': 'Nickname', 'created_str': 'Tweet date',
                   'retweet_count': 'Retweets', 'favorite_count': 'Favorites', 'text': 'Tweet',
                   'hashtags': 'Hashtags', 'expanded_urls': 'URLs', 'mentions': 'Mentions'}
normal_new_cols = {'Tweet content': 'Tweet', 'Favs': 'Favorites', 'RTs': 'Retweets', 
                   'Place (as appears on Bio)': 'Location', 'Tweet language (ISO 639-1)': 'Language'}

troll_users = troll_users.rename(columns=users_new_cols)
troll_tweets = troll_tweets.rename(columns=tweets_new_cols)
normal_tweets = normal_tweets.rename(columns=normal_new_cols)

# Make times formats the same
normal_tweets['Tweet date'] = normal_tweets['Date'] + ' ' + normal_tweets['Hour']
normal_tweets = normal_tweets.drop(['Date', 'Hour'], axis=1)

# Helps with merging
troll_users = troll_users.drop(['User Id', 'Account creation date'], axis=1)
troll_tweets = troll_tweets.drop('User Id', axis=1)

In [7]:
print('Columns of each dataframe after cleaning :\n\nTroll users : {}\n\nTroll tweets : {}\n\nNormal tweets : {}'
      .format(troll_users.columns.values, troll_tweets.columns.values, normal_tweets.columns.values))

Columns of each dataframe after cleaning :

Troll users : ['Location' 'User Name' 'Followers' 'Language' 'Nickname' 'Bio'
 'Following' 'Listed']

Troll tweets : ['Nickname' 'Tweet date' 'Retweets' 'Favorites' 'Tweet' 'Hashtags' 'URLs'
 'Mentions']

Normal tweets : ['User Name' 'Nickname' 'Bio' 'Tweet' 'Favorites' 'Retweets' 'Location'
 'Followers' 'Following' 'Listed' 'Language' 'Tweet date']


### Labeling and merging the datasets

In [8]:
# Merge the troll datasets
trolls = pd.merge(left=troll_tweets, right=troll_users)

In [9]:
trolls.iloc[:2, :]

,Nickname,Tweet date,Retweets,Favorites,Tweet,Hashtags,URLs,Mentions,Location,User Name,Followers,Language,Bio,Following,Listed
0,kathiemrr,2017-02-27 14:54:00,NaN,NaN,#ThingsDoneByMistake kissing auntie in the lips,"[""ThingsDoneByMistake""]",[],[],Atlanta,Kathie,2970.0,en,"Imperfection is beauty, madness is genius and ...",3157.0,22.0
1,kathiemrr,2016-11-28 16:17:36,NaN,NaN,RT @jadedsweetangel: #ToDoListBeforeChristmas ...,"[""ToDoListBeforeChristmas""]",[],[],Atlanta,Kathie,2970.0,en,"Imperfection is beauty, madness is genius and ...",3157.0,22.0


In [10]:
normal_tweets.iloc[:2, :]

,User Name,Nickname,Bio,Tweet,Favorites,Retweets,Location,Followers,Following,Listed,Language,Tweet date
0,Bill Schulhoff,BillSchulhoff,"Husband,Dad,GrandDad,Ordained Minister, Umpire...","Wind 3.2 mph NNE. Barometer 30.20 in, Rising s...",NaN,NaN,"East Patchogue, NY",386.0,705.0,24.0,en,2016-04-16 12:44
1,Daniele Polis,danipolis,"Viagens, geek, moda, batons laranja, cabelos c...",Pausa pro café antes de embarcar no próximo vô...,NaN,NaN,"Grapevine, TX",812.0,647.0,16.0,pt,2016-04-16 12:44


In [11]:
# Put labels
trolls['y'] = pd.Series([1 for x in range(len(trolls))], index=trolls.index)
normal_tweets['y'] = pd.Series([0 for x in range(len(normal_tweets))], index=normal_tweets.index)

In [12]:
# Merge normal and troll data 
tweets = trolls.append(normal_tweets, ignore_index=True)
print(tweets.shape)

(408271, 16)


In [13]:
# Random rows we will use for visualization
rows = [random.randint(0, len(tweets)) for i in range(4)]

In [14]:
tweets.iloc[rows, :]

,Bio,Favorites,Followers,Following,Hashtags,Language,Listed,Location,Mentions,Nickname,Retweets,Tweet,Tweet date,URLs,User Name,y
110324,NaN,NaN,963.0,915.0,[],en,10.0,"St Louis, MO",[],melanymelanin,NaN,RT @trueblacknews: Trump🇷🇺 is as fake as Monic...,2017-01-17 01:59:12,[],̧M̧ȩļa̧ņi̧ņ,1
228568,"Loves cars, good music, great wine, old whisky...",NaN,183.0,342.0,NaN,en,27.0,"Santa Rosa, CA",NaN,guidobrunetti,NaN,Cactus Bear #charlie #teddy #bear #medved #ste...,2016-04-16 03:02,NaN,Thomas,0
88959,"I be trying to chill, I swear do, because I lu...",NaN,908.0,1075.0,"[""strange""]",en,5.0,USA,[],paulinett,NaN,RT @blicqer: ◎ She Noticed Something Unusual O...,2016-10-31 10:27:51,[],Pauline Gunn,1
56786,"Wife, Mother, Patriot, Friend",0.0,2464.0,1930.0,[],en,160.0,USA,"[""kellyannepolls""]",ameliebaldwin,0.0,RT @KellyannePolls: That is a great idea. http...,2016-09-16 14:55:11,"[""https://twitter.com/FreeBeacon/status/776787...",Amelie Baldwin,1


### Extracting hashtags and mentions

In [15]:
def extract_hashtag(string):
    htags = []
    string = str(string)
    for word in string.split():
        if word[0] == '#':
            htags.append(word[1:])
    splits= []
    # Puts space before capital letters and numbers
    for h in htags:
        splits.append(re.sub(r"([A-Z0-9])", r" \1", h)[1:])
    return splits

def extract_mentions(string):
    men = []
    string = str(string)
    for word in string.split():
        if word[0] == '@':
            if word[-1] == ':':
                men.append(word[1:-1])
            else:
                men.append(word[1:])
    return men

# Not implemented yet, takes way too long to run on ~400,000 tweets
def extract_links(string):
    links = []
    string = str(string)
    for word in string.split():
        if word[:4] == 'http':
            try:
                link = urlopen(word).url
                links.append(link)
            except:
                pass
    return links

def extract_website(string):
    string = str(string)
    start = string.find('//') + 2
    if start > 1:
        if string[start:start+3] == 'www':
            start = start + 3
        end = string[start:].find('.') + start
        return string[start:end]
    else:
        return ''
    
def better_date(date):
    date = str(date)
    try: 
        d = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        return int(datetime.now(tz=timezone.utc).timestamp() * 1000)
    except:
        return np.nan

In [16]:
# 8 sec for 100 = 10 hrs for everything : Not viable
start = time.time()
teste = tweets['Tweet'][:100].apply(extract_links)
print(time.time() - start)

9.617864847183228


In [17]:
tweets['Hashtags'] = tweets['Tweet'].apply(extract_hashtag)
tweets['Mentions'] = tweets['Tweet'].apply(extract_mentions)
tweets['Tweet date'] = tweets['Tweet date'].apply(better_date)

In [18]:
tweets['Tweet date'] = tweets['Tweet date'] - min(tweets['Tweet date'])

In [90]:
tweets['Websites'] = tweets['URLs'].apply(extract_website)
tweets['Websites'][tweets['Websites'] != ''][:10]


330       twitter
696     wikileaks
1306      twitter
1504      twitter
1574        paper
1621         vine
2002      twitter
2833      twitter
3340      twitter
3461      twitter
Name: Websites, dtype: object

In [91]:
def clean(string):
    string = str(string)
    string = string.split()
    to_remove = []
    for word in string:
        if word[0] == '#' or word[0] == '@' or word == 'RT' or word[:4] == 'http':
            to_remove.append(word)
    for word in to_remove:
        string.remove(word)
    return ' '.join(string)

In [92]:
tweets['Tweet'] = tweets['Tweet'].apply(clean)

In [93]:
tweets.iloc[rows, :]

,Bio,Favorites,Followers,Following,Hashtags,Language,Listed,Location,Mentions,Nickname,Retweets,Tweet,Tweet date,URLs,User Name,y,Websites
330698,Follow this account for geo-targeted Governmen...,NaN,308.0,279.0,"[ob?, Waukesha,, Government, Hiring]",en,2.0,"Waukesha, WI",[],tmj_mke_govern,NaN,Can you recommend anyone for this Senior Elect...,NaN,NaN,TMJ-MKE Govern Jobs,0,
188597,The best time to plant a tree was 20 years ago...,0.0,841.0,456.0,[],en,18.0,USA,[LPDonovan],javonhidp,0.0,Excellent allocation of resources...,1.521565e+12,"[""https://twitter.com/cahnemily/status/7830733...",Javon Grayson,1,twitter
84135,Black is my religion. If you buy me lipstick w...,NaN,768.0,415.0,[],en,12.0,USA,[BlackGirlNerds],brianaregland,NaN,Toby and Kate tho!,1.521565e+12,[],Briana Ragland,1,
156809,Politics is a circus of hypocrisy. I DO care. ...,NaN,61609.0,20600.0,[3 Words To Regret],en,456.0,USA,[],jenn_abrams,NaN,You are fired,1.521565e+12,[],Jenna Abrams,1,


# Different methods for NLP : One hot encoding and word2vec

In [ ]:
create 1 hot dataframe :
https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas

In [ ]:
Create word2vec dataframe

https://datascience.stackexchange.com/questions/10695/how-to-initialize-a-new-word2vec-model-with-pre-trained-model-weights
    

In [ ]:
Create training and validation set

# Different methods for classification : SVM and K Nearsest Neighbors 

NOTE: The gridsearch objects may require parameters to reduce memory consumption

In [ ]:
Create SVM
#initialize classifier objects for the 1-hot and word2vec encodings
svc_1_hot = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

svc_word2vec = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', random_state=None)

#initialize parameters

#penalty parameter
C_list =[10**(1-i) for i in range(6)]
#kernel function - poly:polynomial
kernel_list =['linear','rbf','sigmoid','poly']
param_dic_SVM = {'C':C_list, 'kernel':kernel_list}

In [ ]:
Train SVM using GridSearch on one-hot
save results in list

clf_svm_1 = GridSearchCV(estimator, param_dict_SVM, cv=5, refit = True)


In [ ]:
Train SVM using Gridsearch on w2v
save results in list

clf_svm_2 = GridSearchCV(estimator, param_dict_SVM, cv=5, refit = True)


In [ ]:
Create KNN

knn_one_hot = neighbors.KNeighborsClassifier(n_jobs=-1)

knn_word2vec = neighbors.KNeighborsClassifier(n_jobs=-1)

#initialize parameters

#number of nearest neighbors to consider: 3-10
n_neighbors_list = [i+3 for i in range(8)]
#weighting of neighbors, whether they all get an equal vote or are weighted by distance
weights_list = ['uniform','distance']
param_dic_knn = {'n_neighbors':n_neighbors_list,'weights':weights_list}


In [ ]:
Train KNN using GridSearch on one-hot
save results in list

clf_knn_1 = GridSearchCV(knn_one_hot, param_dict_knn, cv=5, refit = True)

In [ ]:
Train KNN using Gridsearch on w2v
save results in list

clf_knn_2 = GridSearchCV(knn_word2vec, param_dict_knn, cv=5, refit = True)

# Results

In [ ]:
Plot and present results